In [ ]:
# !pip install torchvision

In [8]:

import numpy as np
import autokeras as ak
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_tensor
from torchvision.transforms import v2

In [2]:
data_dir = 'Dog Breed Dataset'

# Use ImageDataGenerator to load and preprocess the images
datagen = ImageDataGenerator(rescale = 1. / 255)

# Define the data generators for training, validation, and testing
train_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/train',
    target_size = (256, 256),
    batch_size = 16,
    class_mode = 'categorical'
)


test_generator = datagen.flow_from_directory(
    directory = f'{data_dir}/test',
    target_size = (256, 256),
    batch_size = 1,
    class_mode = 'categorical',
    shuffle = False
)

Found 1443 images belonging to 29 classes.
Found 374 images belonging to 29 classes.


In [9]:


# Initialize empty lists for training and test data and labels
x_train, y_train = [], []
x_test, y_test = [], []

# Load training data in batches
for _ in range(len(train_generator)):
    batch_x, batch_y = next(train_generator)
    x_train.append(batch_x)
    y_train.append(batch_y)

# Load test data in batches
for _ in range(len(test_generator)):
    batch_x, batch_y = next(test_generator)
    x_test.append(batch_x)
    y_test.append(batch_y)



In [10]:
# Concatenate the training and test data
x_train = np.concatenate(x_train)
x_test = np.concatenate(x_test)

# Concatenate the labels and then convert to integer labels
y_train = np.concatenate(y_train)
y_train = np.argmax(y_train, axis=1)

y_test = np.concatenate(y_test)
y_test = np.argmax(y_test, axis=1)


In [11]:
clf = ak.ImageClassifier(
    max_trials = 5,
    objective = 'val_accuracy',
    overwrite = True,
)

# Search for the best model architecture
clf.fit(x_train, y_train, epochs = 10)

2024-04-20 13:46:13.263355: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-20 13:46:13.266855: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-20 13:46:13.266946: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
vanilla           |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
False             |False             |image_block_1/augment
3                 |3                 |image_block_1/conv_block_1/kernel_size
1                 |1                 |image_block_1/conv_block_1/num_blocks
2                 |2                 |image_block_1/conv_block_1/num_layers
True              |True              |image_block_1/conv_block_1/max_pooling
False             |False             |image_block_1/conv_block_1/separable
0.25              |0.25              |image_block_1/conv_block_1/dropout
32                |32                |image_block_1/conv_block_1/filters_0_0
64                |64                |image_block_1/conv_block_1/filters_0_1
flatten           |flatten           |classification_head_1/spatial_reduction_1/reduction_type
0.5               |0.5          

W0000 00:00:1713591978.587028    3669 asm_compiler.cc:252] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
W0000 00:00:1713591978.587076    3669 asm_compiler.cc:255] Used ptxas at /usr/bin/ptxas
2024-04-20 13:46:18.587156: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-04-20 13:46:18.739341: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-04-20 13:46:18.739371: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-04-20 13:46:18.739389: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to c

Epoch 1/10


2024-04-20 13:46:19.246449: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
I0000 00:00:1713591980.490575    7961 service.cc:145] XLA service 0x708f1c0041b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713591980.490703    7961 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-04-20 13:46:20.625165: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-20 13:46:20.775216: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


In [8]:
results = clf.evaluate(x_test, y_test)
print(results)
print("Test accuracy:", results[1])

2/2 [==============================] - 2s 631ms/step - loss: 1.1038e-08 - accuracy: 1.0000
[1.1037893266063747e-08, 1.0]
Test accuracy: 1.0


In [9]:
from sklearn.metrics import classification_report

In [10]:
y_pred = clf.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, accuracy, and F1 score
report = classification_report(y_test, y_pred_classes)
print(report)

2/2 [==============================] - 2s 867ms/step
              precision    recall  f1-score   support

           0       0.33      1.00      0.50        18
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        18

    accuracy                           0.33        54
   macro avg       0.11      0.33      0.17        54
weighted avg       0.11      0.33      0.17        54



c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:

clf.export_model().save("Dog Breed Model")